### 8.4.2 Sending qubits in superposition
Another attempt to share secret key.

In [ ]:
using ImageShow
using Random
using StrangelyDisplayed
using StrangelyQuantum

A "secret" key is generated by randomly putting `key_length` qubits in either the
$|0\hspace{-0.25em}>$ or the $|1\hspace{-0.25em}>$ state. Before sending the qubits, a Hadamard gate is
applied so that the qubits are in a superposition state.
Before the receiver measures them, another Hadamard gate is applied,
so that the qubit returns to the original state.
An eavesdropper can still spy on the communication if this approach is known.

In [ ]:
function superpos_example(key_length::Integer)
    random = Xoshiro()
    # Create a key containing random bits.
    aliceBits = [rand(random, Bool) for _ = 1:key_length]

    simulator = SimpleQuantumExecutionEnvironment()
    program = Program(key_length)
    prepareStep = Step()
    superPositionStep = Step()
    superPositionStep2 = Step()
    measureStep = Step()
    for i = 1:key_length
        # Initialize the qubits according to the random bits.
        # A random bit of 0 will lead to a |0⟩ qubit;
        # a random bit of 1 will lead to a |1⟩ qubit.
        aliceBits[i] && addGate(prepareStep, X(i))
        # perform a Hadamard transformation to bring the
        # qubit into a superposition and send it over the network.
        addGate(superPositionStep, Hadamard(i))
        # Perform a second Hadamard transformation on the received qubit.
        addGate(superPositionStep2, Hadamard(i))
        # Measure the qubit.
        addGate(measureStep, Measurement(i))
    end

    addStep(program, prepareStep)
    addStep(program, superPositionStep)
    addStep(program, superPositionStep2)
    addStep(program, measureStep)

    result = runProgram(simulator, program)
    qubit = getQubits(result)

    measurement = Vector{Int}(undef, key_length)
    bobBits = Vector{Bool}(undef, key_length)
    for i = 1:key_length
        measurement[i] = measure(qubit[i])
        bobBits[i] = measurement[i] == 1
        println(
            "Alice sent ",
            aliceBits[i] ? '1' : '0',
            " and Bob received ",
            bobBits[i] ? '1' : '0',
        )
    end

    return program
end

Execute the program with a key length of 4:

In [ ]:
program = superpos_example(4)

The program looks like this:

In [ ]:
drawProgram(program)